# Introduction to Genetic Algorithms
Genetic Algorithms (GAs) are a class of evolutionary algorithms that mimic the process of natural selection to solve complex optimization problems. They operate on a population of potential solutions, applying the principles of selection, crossover, and mutation to evolve towards better solutions over generations.

At the heart of GAs is the concept of survival of the fittest. Initially, a diverse population of solutions is generated. Each member of this population, often referred to as an individual or chromosome, represents a possible solution to the problem at hand. These solutions are evaluated using a fitness function, which measures how well they solve the problem.

The evolutionary process begins by selecting the fittest individuals to reproduce. Crossover, or recombination, then combines parts of two selected solutions to create offspring, introducing new solutions into the population. Mutation randomly alters parts of a solution, introducing variability. Over successive generations, this process of selection, crossover, and mutation drives the population towards increasingly effective solutions.

GAs are widely used for their ability to find high-quality solutions to optimization problems where traditional methods may struggle, especially in complex or poorly-understood problem spaces.

## Genetic Algorithm for Adaptive Defense Strategy Optimization

In the rapidly evolving landscape of modern military operations, the strategic allocation of defense resources and the planning of operations play pivotal roles in ensuring national security and operational success. The complexity of these decisions, influenced by a myriad of unpredictable factors such as enemy tactics, terrain, and the availability of resources, necessitates a highly adaptable and efficient approach to defense strategy formulation. This scenario introduces the application of Genetic Algorithms (GAs) in optimizing adaptive defense strategies, a novel approach that transcends traditional planning methods by incorporating the principles of evolution and natural selection.

Genetic algorithms offer a robust framework for exploring a vast search space of possible strategies, each represented by a unique combination of tactical decisions, resource allocations, and operational plans. By defining a fitness function that evaluates the effectiveness of a strategy based on criteria such as mission success probability, resource efficiency, and casualty minimization, GAs can iteratively improve a population of strategies. Through the processes of selection, crossover, and mutation, each generation of strategies evolves to better meet the demands of the operational environment, thereby converging towards an optimal or near-optimal solution.

The application of GAs in this context allows for the dynamic adaptation of defense strategies in response to changing conditions on the ground. For instance, as new intelligence is received or as the operational environment shifts, the GA can quickly adjust, testing and evolving new strategies that respond to these changes. This capability is particularly valuable in scenarios where human decision-makers may not be able to rapidly process all available information or foresee the complex interactions between different elements of a strategy.

Moreover, the use of GAs in military strategy optimization facilitates a deeper understanding of the trade-offs and synergies between various components of a defense plan. By analyzing the evolution of strategies over successive generations, military planners can gain insights into which tactics and resource allocations are most critical for mission success under different conditions. This not only aids in the development of more effective immediate strategies but also contributes to long-term strategic planning and the development of flexible, resilient military doctrines.

### Sets and Indices
- $A$: Set of possible actions or tactical decisions, indexed by $a$.
- $R$: Set of resources available for allocation, indexed by $r$.
- $S$: Set of scenarios or operational environments, indexed by $s$.

### Parameters
- $P_s$: Probability of occurrence for scenario $s$.
- $E_{a,r,s}$: Effectiveness of action $a$ using resource $r$ in scenario $s$, a measure combining success probability, resource efficiency, and other relevant metrics.
- $C_r$: Capacity or availability of resource $r$.
- $D_{a,r}$: Demand or requirement of resource $r$ for action $a$.
- $W_s$: Weight or importance of scenario $s$, reflecting strategic priorities.

### Decision Variables
- $x_{a,r}$: Binary variable, 1 if action $a$ is selected with resource $r$, 0 otherwise.
- $y_s$: Real-valued variable representing the overall strategy effectiveness in scenario $s$.

### Objective Function
Maximize the weighted effectiveness of the defense strategy across all scenarios:
$$\max \sum_{s \in S} W_s \cdot y_s$$

### Constraints
1. **Strategy Effectiveness Constraint**: Define the effectiveness of the strategy in each scenario based on selected actions and allocated resources.
   $$y_s = \sum_{a \in A} \sum_{r \in R} E_{a,r,s} \cdot x_{a,r} \quad \forall s \in S$$

2. **Resource Allocation Constraints**: Ensure that the allocation of each resource does not exceed its capacity.
   $$\sum_{a \in A} D_{a,r} \cdot x_{a,r} \leq C_r \quad \forall r \in R$$

3. **Action Selection Constraints**: Ensure that actions are selected in a manner that respects resource availability and operational requirements.
   $$x_{a,r} \in \{0, 1\} \quad \forall a \in A, \forall r \in R$$

4. **Scenario Probability Constraints**: Ensure that the strategy considers the probability of occurrence for each scenario.
   $$\sum_{s \in S} P_s \cdot y_s \leq 1$$


In [1]:
import random
import numpy
from deap import base, creator, tools, algorithms

# Adjusting the individual creation to match the expected structure
NUM_ACTIONS = 5
NUM_RESOURCES = 3
NUM_SCENARIOS = 4
MAX_RESOURCE_CAPACITY = 100  # Max capacity for each resource

# Assuming a simplified model for demonstration
effectiveness_scores = [[[random.uniform(0, 1) for _ in range(NUM_SCENARIOS)] for _ in range(NUM_RESOURCES)] for _ in range(NUM_ACTIONS)]
scenario_probabilities = [random.uniform(0, 1) for _ in range(NUM_SCENARIOS)]
scenario_weights = [random.uniform(1, 3) for _ in range(NUM_SCENARIOS)]

# Fitness Function
def evaluate(individual):
    # Reshape the individual to match the expected structure
    strategy = numpy.reshape(individual, (NUM_ACTIONS, NUM_RESOURCES))
    
    total_effectiveness = 0
    resource_usage = [0] * NUM_RESOURCES
    
    for action_idx in range(NUM_ACTIONS):
        for resource_idx in range(NUM_RESOURCES):
            allocation = strategy[action_idx][resource_idx]
            resource_usage[resource_idx] += allocation
            for scenario_idx in range(NUM_SCENARIOS):
                total_effectiveness += (effectiveness_scores[action_idx][resource_idx][scenario_idx] * allocation * scenario_weights[scenario_idx])
    
    # Penalize for exceeding resource capacity
    penalty = sum(max(0, usage - MAX_RESOURCE_CAPACITY) for usage in resource_usage)
    
    return total_effectiveness - penalty,

# DEAP Setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, MAX_RESOURCE_CAPACITY / (NUM_ACTIONS * NUM_RESOURCES))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=NUM_ACTIONS * NUM_RESOURCES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=20, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Running the Genetic Algorithm
def main():
    population = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)
    
    return population, stats, hof

if __name__ == "__main__":
    main()


gen	nevals	avg    	std    	min    	max  
0  	50    	195.341	30.1487	138.127	278.4
1  	31    	223.422	59.845 	110.053	531.705
2  	33    	273.523	79.7786	184.812	577.25 
3  	25    	335.865	114.009	79.1148	614.503
4  	38    	431.761	111.03 	209.949	679.888
5  	31    	520.036	81.0795	329.445	720.512
6  	31    	588.747	76.8113	389.192	818.417
7  	28    	624.356	81.5837	359.283	787.238
8  	30    	701.153	74.7044	505.279	846.812
9  	30    	765.938	85.8283	491.325	950.351
10 	28    	822.568	75.5834	535.37 	1010.31
11 	15    	888.812	79.655 	795.269	1260.71
12 	28    	947.368	81.53  	802.332	1260.71
13 	34    	1015.95	70.1119	876.17 	1171.84
14 	24    	1068.1 	69.53  	947.11 	1216.32
15 	25    	1122.75	98.1326	820.138	1332.8 
16 	25    	1202.63	82.1222	1010.31	1395.16
17 	27    	1269.2 	110.93 	940.184	1499   
18 	29    	1364.88	109.646	1047.63	1627.65
19 	33    	1459.61	118.979	1059.98	1675.97
20 	26    	1558.9 	80.5668	1395.19	1710.48
21 	32    	1627.4 	70.9787	1483.83	1814.21
22 	33    	1703